# Project 1

The goal of this assignment is to help you build your intuition about recommender systems, with a basic soup to nuts implementation coded “from scratch.”

Your task is to build a very basic recommender system, first by writing your own functions, then by replacing those functions with those provided in an R Package or a Python library (such as scikit-learn).

* You should very briefly first describe the recommender system that you’re going to build out from a business perspective, e.g. “This system recommends movies to users.”
* You can find a dataset, or build out your own toy dataset and load into (for example) an R or pandas dataframe, a Python dictionary or list of lists, (or other data structure of your choosing).
* You can use either collaborative filtering, or a hybrid of content management and collaborative filtering. 
* You are encouraged to hand code at least your similarity function.
* After you have built out your own code base, create an alternate version using packages or libraries.  Compare the results and performance.
* You are also encouraged to think about how to best handle missing data.
* Your code should be turned in an RMarkdown file or a Jupyter notebook, and posted to Github.


** Requires the Jupyter-Scala language Kernel, available from: (https://github.com/alexarchambault/jupyter-scala)[https://github.com/alexarchambault/jupyter-scala]

In [51]:
classpath.add( "org.apache.spark" %% "spark-core" % "1.6.1",
             "org.apache.spark" %% "spark-mllib" % "1.6.1",
              "org.apache.spark" %% "spark-sql" % "1.6.1")

0 new artifact(s)


# Response

## The Recommender System

As I'm farily new to Spark and the whole data manipulation world in Scala, let's keep the problem simple. This is a system that recommends movies to users based on the dataset collected by the class survey.

As part of this exercise, I will produce a manual similarity function and compare the performance against the collaborative filtering library in Spark

## The Code

### Firing up a Spark Context

In [52]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType,StructField,StringType,IntegerType}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType,StructField,StringType,IntegerType}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

In [53]:


val conf = new SparkConf()
  .setAppName("week1-EstimatePi")
  .setMaster("local") 
val sc = new SparkContext(conf)


conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@582f9904
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2d1c95ef

### Data Loading and Transformations

The objective here is to:

* Load the `MovieRatings.csv` file
* Transform into Zero filled matrix
* Transform into Long-format data structure


In [54]:
// Read the CSV file
val csv = 
    sc
        .textFile("MovieRatings.csv")
        .map(line => 
             line
                 .replaceAll(",$",", ")
                 .split(",")
                 .map(t => t.trim)
            )
csv.collect


csv: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at Main.scala:29
res53_1: Array[Array[String]] = Array(
  Array(
    "Critic",
    "CaptainAmerica",
    "Deadpool",
    "Frozen",
    "JungleBook",
    "PitchPerfect2",
    "StarWarsForce"
  ),
  Array("Burton", "", "", "", "4", "", "4"),
  Array("Charley", "4", "5", "4", "3", "2", "3"),
  Array("Dan", "", "5", "", "", "", "5"),
  Array("Dieudonne", "5", "4", "", "", "", "5"),
  Array("Matt", "4", "", "2", "", "2", "5"),
  Array("Mauricio", "4", "", "3", "3", "4", ""),
  Array("Max", "4", "4", "4", "2", "2", "4"),
  Array("Nathan", "", "", "", "", "", "4"),
  Array("Param", "4", "4", "1", "", "", "5"),
  Array("Parshu", "4", "3", "5", "5", "2", "3"),
...

#### Transforming into Zero-filled Matrix

In [55]:
//val movies = sc.parallelize(csv.first)
val movies = csv.first.splitAt(1)._2
val critics = csv.collect.map(_(0)).splitAt(1)._2

// let's also make a parallelized version of those
val moviesPar = sc.parallelize(movies).zipWithIndex
val criticsPar = sc.parallelize(critics).zipWithIndex

movies: Array[String] = Array(
  "CaptainAmerica",
  "Deadpool",
  "Frozen",
  "JungleBook",
  "PitchPerfect2",
  "StarWarsForce"
)
critics: Array[String] = Array(
  "Burton",
  "Charley",
  "Dan",
  "Dieudonne",
  "Matt",
  "Mauricio",
  "Max",
  "Nathan",
  "Param",
  "Parshu",
  "Prashanth",
  "Shipra",
  "Sreejaya",
  "Steve",
  "Vuthy",
  "Xingjia"
)
moviesPar: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[4] at zipWithIndex at Main.scala:38
criticsPar: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[6] at zipWithIndex at Main.scala:41

In [56]:
val zeroFilledMatrix = 
    csv
        .collect
        .filterNot(r => r(1) == movies(0)) // filter out first row
        .map(r => r.filterNot(value => critics contains value)) // map function that returns the record minus the critic name
        .map(r => r.map(value => if (value == "") 0.00 else value.toDouble))

// and now parallelize it
val zeroFilledMatrixPar = sc.parallelize(zeroFilledMatrix)

// now, let's convert it into a linalg matrix so we can perform linear algebra operations on it
val criticMoviesMatrix = new RowMatrix(zeroFilledMatrixPar.map(line => Vectors.dense(line)))


// also, let's have a transposed version of it:
val dataTransposed =  sc.parallelize(zeroFilledMatrix.toSeq.transpose)

val moviesCriticMatrix = new RowMatrix(dataTransposed.map(line => Vectors.dense(line.toArray)))


zeroFilledMatrix: Array[Array[Double]] = Array(
  Array(0.0, 0.0, 0.0, 4.0, 0.0, 4.0),
  Array(4.0, 5.0, 4.0, 3.0, 2.0, 3.0),
  Array(0.0, 5.0, 0.0, 0.0, 0.0, 5.0),
  Array(5.0, 4.0, 0.0, 0.0, 0.0, 5.0),
  Array(4.0, 0.0, 2.0, 0.0, 2.0, 5.0),
  Array(4.0, 0.0, 3.0, 3.0, 4.0, 0.0),
  Array(4.0, 4.0, 4.0, 2.0, 2.0, 4.0),
  Array(0.0, 0.0, 0.0, 0.0, 0.0, 4.0),
  Array(4.0, 4.0, 1.0, 0.0, 0.0, 5.0),
  Array(4.0, 3.0, 5.0, 5.0, 2.0, 3.0),
  Array(5.0, 5.0, 5.0, 5.0, 0.0, 4.0),
  Array(0.0, 0.0, 4.0, 5.0, 0.0, 3.0),
  Array(5.0, 5.0, 5.0, 4.0, 4.0, 5.0),
  Array(4.0, 0.0, 0.0, 0.0, 0.0, 4.0),
  Array(4.0, 5.0, 3.0, 3.0, 3.0, 0.0),
  Array(0.0, 0.0, 5.0, 5.0, 0.0, 0.0)
)
zeroFilledMatrixPar: org.apache.spark.rdd.RDD[Array[Double]] = ParallelCollectionRDD[7] at parallelize at Main.scala:47
criticMoviesMatrix: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@2cf2ec0c
dataTransposed: org.apache.spark.rdd.RDD[Seq[Double]] = ParallelCollecti

#### Transforming into a Long-format data structure

For practical purposes, we'll do an index-based long format, meaning that the string names will be substituted for an index

In [57]:
val longFormat = 
    csv
        .collect
        .filterNot(r => r(1) == movies(0)) // filter out first row
        .flatMap(r=> (1 to movies.length-1).map(i=> (r(0),movies(i-1),r(i)))) // pivot each column sothat we have: (user,movie,rating)
        .filter(r=> r._3 !="") // filter out those unrated movies
        .map(r=> (critics.indexOf(r._1),movies.indexOf(r._2),r._3.toDouble)) // convert the remaining rating to a double
                           
val ratingsLong = sc.parallelize(longFormat)

longFormat: Array[(Int, Int, Double)] = Array(
  (0, 3, 4.0),
  (1, 0, 4.0),
  (1, 1, 5.0),
  (1, 2, 4.0),
  (1, 3, 3.0),
  (1, 4, 2.0),
  (2, 1, 5.0),
  (3, 0, 5.0),
  (3, 1, 4.0),
  (4, 0, 4.0),
  (4, 2, 2.0),
  (4, 4, 2.0),
  (5, 0, 4.0),
  (5, 2, 3.0),
  (5, 3, 3.0),
  (5, 4, 4.0),
  (6, 0, 4.0),
  (6, 1, 4.0),
  (6, 2, 4.0),
...
ratingsLong: org.apache.spark.rdd.RDD[(Int, Int, Double)] = ParallelCollectionRDD[11] at parallelize at Main.scala:42

## Model Building - Manual Recommendations System


### Critic-Critic Similarity Model

Let's now build a User-User similarity model based on the cosine distance

In [58]:
val userUserCosineDistance = moviesCriticMatrix.columnSimilarities()

val userUserSimilarities = userUserCosineDistance
  .entries
  .map {
    case MatrixEntry(i, j, u) => (i, j, u) }
  .collect
  .map(r => Seq(critics(r._1.toInt), critics(r._2.toInt), r._3.toDouble))
  .sortBy(-_(2).asInstanceOf[Double])

userUserCosineDistance: org.apache.spark.mllib.linalg.distributed.CoordinateMatrix = org.apache.spark.mllib.linalg.distributed.CoordinateMatrix@44033cf3
userUserSimilarities: Array[Seq[Any]] = Array(
  List(Dieudonne, Param, 0.9859249803487347),
  List(Max, Sreejaya, 0.9847319278346619),
  List(Charley, Max, 0.9811873171500672),
  List(Charley, Sreejaya, 0.9792633226865932),
  List(Charley, Prashanth, 0.9610484599102903),
  List(Parshu, Prashanth, 0.9600666937386864),
  List(Parshu, Sreejaya, 0.955672134494952),
  List(Charley, Parshu, 0.9474847084398104),
  List(Max, Prashanth, 0.9410294354946785),
  List(Max, Parshu, 0.9296599791147713),
  List(Prashanth, Sreejaya, 0.9293555142631518),
  List(Charley, Vuthy, 0.927771250724491),
  List(Matt, Steve, 0.9091372900969895),
  List(Shipra, Xingjia, 0.8999999999999999),
  List(Sreejaya, Vuthy, 0.8866206949335731),
  List(Dieudonne, Steve, 0.870388279778489),
  List(Max, Param, 0.866578244826242),
  List(Parshu, Vuthy, 0.8661218807416571),
  

### Putting Together a Recommender System

Let's try the following strategy:

* For a given user, find simiar users. A similar user is another user with a cosine distance of at least 0.6
* Make a movie rating recommendation based on the average of the similar user-group

First, let's Generate Movie ratings for all critics:

In [59]:
def recommendMoviesForUser(user:String,userUserSimilarities:Array[Seq[Any]],ratings:Array[Array[Double]])={
    val similarUsers = userUserSimilarities
        .filter(r=> r(0) == user || r(1)==user).map(r=>(if (r(1)==user) r(0) else r(1), r(2).asInstanceOf[Double]))
        .filter(_._2>0.6)
        .map(r=> (critics.indexOf(r._1),r._2))
        .sortBy(-_._2)

    val similarRatings = ratings
        .zipWithIndex // add user indexes
        .filter(r=>similarUsers.map(_._1) contains r._2) // filter out based on the index
        .map(_._1) // revert back to the array
    val meanRating = (0 to (movies.length-1)).map{
        m => 
            val nonZeroRatings = similarRatings.map(c => c(m)).filter(_>0)
            nonZeroRatings.sum/nonZeroRatings.length
    }.toArray
    meanRating
}


val predictedCriticRatings = critics.map(c=>recommendMoviesForUser(c,userUserSimilarities,zeroFilledMatrix))

defined function recommendMoviesForUser
predictedCriticRatings: Array[Array[Double]] = Array(
  Array(4.0, 3.0, 4.5, 5.0, 2.0, 3.3333333333333335),
  Array(
    4.333333333333333,
    4.375,
    3.5555555555555554,
    3.857142857142857,
    2.8333333333333335,
    4.333333333333333
  ),
  Array(4.4, 4.4, 3.5, 3.0, 2.6666666666666665, 4.333333333333333),
  Array(4.25, 4.428571428571429, 3.7142857142857144, 3.8, 2.4, 4.2),
  Array(
    4.333333333333333,
    4.285714285714286,
    3.857142857142857,
    3.6666666666666665,
    2.8,
    4.111111111111111
  ),
...

Let's now calculate the mean squared error for this model

In [60]:
val MSE = (0 to critics.length-1).map{ 
    c => 
        val row = (0 to movies.length-1).map{ 
            m =>
                val err = predictedCriticRatings(c)(m) - zeroFilledMatrix(c)(m)
                err * err
        }
        row.sum / row.length
}
MSE.sum/MSE.length

MSE: collection.immutable.IndexedSeq[Double] = Vector(
  8.449074074074074,
  0.6510305125136474,
  8.087592592592593,
  5.897015306122449,
  6.133667590492988,
  6.671012849584277,
  0.8786625514403292,
  9.922453703703702,
  4.949259259259259,
  1.3704329386075418,
  2.003435715335517,
  7.851481481481482,
  1.0986684565801628,
  7.296342592592592,
  2.957535903250189,
  8.578703703703704
)
res59_1: Double = 5.174773076958407

### Querying the Models

* User: Who should Mauricio go out to the movies with?

In [61]:
val user= "Mauricio"
userUserSimilarities
    .filter(r=> r(0) == user || r(1)==user).map(r=>(if (r(1)==user) r(0) else r(1), r(2).asInstanceOf[Double]))
    .filter(_._2>0.75)
    .sortBy(-_._2)

user: String = "Mauricio"
res60_1: Array[(Any, Double)] = Array(
  (Parshu, 0.8140806303599618),
  (Vuthy, 0.7888934916555406),
  (Sreejaya, 0.7754763931697963)
)

* What movies should Mauricio watch?

In [62]:
recommendMoviesForUser("Mauricio",userUserSimilarities,zeroFilledMatrix).zipWithIndex.map(r => (movies(r._2),r._1)).sortBy(-_._2)

res61: Array[(String, Double)] = Array(
  ("Deadpool", 4.5),
  ("CaptainAmerica", 4.285714285714286),
  ("Frozen", 4.0),
  ("StarWarsForce", 4.0),
  ("JungleBook", 3.6666666666666665),
  ("PitchPerfect2", 2.5)
)

## Using Collaborative Filtering ALS Model

Based on: http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html

In [63]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating

val ratingsALS = ratingsLong.map(r=>Rating(r._1, r._2, r._3))

// Build the recommendation model using ALS
val rank = 10
val numIterations = 10
val model = ALS.train(ratingsALS, rank, numIterations, 0.01)


import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating
ratingsALS: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[17] at map at Main.scala:35
rank: Int = 10
numIterations: Int = 10
model: org.apache.spark.mllib.recommendation.MatrixFactorizationModel = org.apache.spark.mllib.recommendation.MatrixFactorizationModel@6b9230e

### Querying the ALS Model

Let's get:

* Recommend 5 movies for Mau

In [66]:
//What movies 
model.recommendProducts(5,critics.indexOf("Mauricio")).map(r => (critics(r.user), movies(r.product), r.rating)) 

model.recommendUsersForProducts(3).collect.flatMap(m=>m._2.map(r=>(movies(m._1),critics(r.user),r.rating)))

model.recommendProductsForUsers(3).collect.flatMap(m=>m._2.map(r=>(movies(r.product),critics(m._1),r.rating)))


: 

In [ ]:
sc.stop

### Model Performance

In [65]:
// Evaluate the model on rating data
val criticsMovies = ratingsALS.map { case Rating(critic, movie, rate) =>
  (critic, movie)
}
val predictions =
  model.predict(criticsMovies).map { case Rating(critic, movie, rate) =>
    ((critic, movie), rate)
  }
val ratesAndPredictions = ratingsALS.map { case Rating(critic, movie, rate) =>
  ((critic, movie), rate)
}.join(predictions)
val MSE = ratesAndPredictions.map { case ((critic, movie), (r1, r2)) =>
  val err = (r1 - r2)
  err * err
}.mean()

: 

# Conclusion

* Clearly the ALS model (MSE=9.456951543917609E-5) beats the manual model (MSE=5.174773076958407)
* Alternative recommendation strategies can be explored such as nearest neighbors
